# The Political Climate of the USA

How much is public opinion on climate change and the environment influenced by the views of the current president? 

In the past ~18 years, the US has seen three different presidents with differing views on climate change. President Bush (2001-2009) was a climate change denier, as is current president Donald Trump. President Obama (2009-2017), however, was proponent of climate change. All three presidents had public policies in place consistent with the views above. 

President Bush "asked the National Academy of Sciences (NAS) to review the findings of the Intergovernmental Panel on Climate Change (IPCC) and provide further assessment of climate science’s “certainties and uncertainties.”3" Furthermore, he "expressed disdain for a State Department report6 to the United Nations that pointed to a clear human role in the accumulation of heat-trapping gases and detailed the likely negative consequences of climate change; the president called it “a report put out by the bureaucracy.”7" (https://www.ucsusa.org/our-work/center-science-and-democracy/promoting-scientific-integrity/manipulation-of-global.html). 

In contrast, President Obama called climate change "the greatest threat to future generations" in his 2015 State of the Union Address and implemented the Global Climate Change Initiative.

Meanwhile, President Trump has already left the Paris climate agreement, [weakened methane regulation](https://www.nytimes.com/2019/08/29/climate/epa-methane-greenhouse-gas.html?module=inline), frozen fuel efficiency standards and [revoked state rights to set their own rules](https://www.washingtonpost.com/climate-environment/trump-administration-to-revoke-californias-power-to-set-stricter-auto-emissions-standards/2019/09/17/79af2ee0-d97b-11e9-a688-303693fb4b0b_story.html?noredirect=on), [eliminated the clean power plan](https://www.washingtonpost.com/climate-environment/trump-administration-to-revoke-californias-power-to-set-stricter-auto-emissions-standards/2019/09/17/79af2ee0-d97b-11e9-a688-303693fb4b0b_story.html?noredirect=on) and more.

In this project I look at how these public policies and presidential attitudes do (or do not) affect public opinion on climate change. Rather than relying on climate change polls, I choose to look at behavior of people in the US. I compare residential energy usage, electric and hybrid car sales, public transportation, consumption of meat, waste produced and recycled, and public transportation useage.

Some of these data sets go back as far as the 1970s, but for the purposes of this project, I'm more interested in 2001 to present. Data on Plug-In electric vehicles is the most restrictive, since the number of sales has only become significant recently; this set begins in 2011 and can only be used to compare President Obama's administration to President Trump's. The waste set has 2000, 2005, and 2010, but nothing in between. Furthermore, several of the sets are missing more recent years. I intend to search for more up-to-date data in the future. For now I consider all sets from 2002 onward. I choose to cut at 2002 instead of 2001 because that's where the public transportation data begin and removing one year of Bush's presidency is unlikely to affect any conclusions.

In [1]:
import pandas as pd
import numpy as np
from itertools import compress
from functools import reduce

## Data Sets

Electricity Use by Year by State: https://www.eia.gov/electricity/data.php#sales 
Plug-in and HEV sales by year by model: https://afdc.energy.gov/data/search?q=hybrid+sales
Number of Alternate-Fuel Fueling Stations: www.afdc.energy.gov/afdc/fuels/stations_counts.html
Total Car Sales: https://fred.stlouisfed.org/series/TOTALSA 
Public Transportation: https://www.transit.dot.gov/ntd/data-product/monthly-module-adjusted-data-release 
Meat Consumption: https://www.ers.usda.gov/data-products/livestock-meat-domestic-data/livestock-meat-domestic-data/ 
Waste and Recycling: https://catalog.data.gov/dataset/sustainable-materials-management-smm-materials-and-waste-management-in-the-united-states-key-f 
Total Population: https://fred.stlouisfed.org/series/POPTOTUSA647NWDB 

In [33]:
fuel=pd.read_excel("data/alt_fueling_stations.xlsx", header=2, usecols=[1,10],skipfooter=20)
hev=pd.read_excel("data/hev_sale.xlsx", header=2, usecols=[1,68],skipfooter=19)
pev=pd.read_excel("data/pev_sales.xlsx", header=2,skipfooter=10)
cars=pd.read_csv("data/total_car_sales.csv")

meat=pd.ExcelFile("data/meat_consumption.xlsx")
red_meat=pd.read_excel(meat,sheet_name="WASDE_TotalRedMeat-Full",header=2, usecols=[0,1,13],names=["red_meat_per_capita"],skipfooter=5)
white_meat=pd.read_excel(meat,sheet_name="WASDE_TotalPoultry-Full",header=2, usecols=[0,1,13],names=["poultry_per_capita"],skipfooter=5)

waste=pd.read_excel("data/waste_recycling.xlsx",skipfooter=3,index_col=0)
pop=pd.read_csv("data/pop_tot.csv")

elec=pd.read_excel("data/electricity.xlsx", header=1)

public_trans = pd.ExcelFile("data/public_trans.xlsx")
cols = [i for i in range(219) if i not in [0,1,2,3,4,5,6,8]]
UPT=pd.read_excel(public_trans,sheet_name="UPT",usecols=cols)
VRM=pd.read_excel(public_trans,sheet_name="VRM",usecols=cols)
VRH=pd.read_excel(public_trans,sheet_name="VRH",usecols=cols)

In [35]:
fuel=fuel.rename(columns={'Year':'year','Total***':'alt_fuel_stations'})
fuel=fuel[fuel['year']>=2002]
fuel=fuel.reset_index(drop=True)

In [36]:
pop_year=[]
for date in pop['DATE']:
           pop_year=pop_year+[int(date[0:4])]
pop['year']=pop_year
pop=pop.drop(columns=['DATE'])
pop=pop.rename(columns={'POPTOTUSA647NWDB':'tot_population'})
pop=pop[['year','tot_population']]
pop=pop[pop['year']>=2002]
pop=pop.reset_index(drop=True)

In [37]:
years=[x[0] for x in red_meat.index]
i=~np.isnan(years)
years=list(compress(years,i))

red_meat_per_capita=[]
poultry_per_capita=[]
for (i,(year,quart)) in enumerate(red_meat.index):
    if quart=='Yr Jan-Dec':
        red_meat_per_capita=red_meat_per_capita+[red_meat['red_meat_per_capita'].iloc[i]]
        poultry_per_capita=poultry_per_capita+[white_meat['poultry_per_capita'].iloc[i]]
    

meat=pd.DataFrame({'year':years,'red_meat':red_meat_per_capita,'white_meat':poultry_per_capita})
meat = meat[meat['year']>=2002]
meat=meat.reset_index(drop=True)

In [38]:
car_year=[]

for date in cars['DATE']:
           car_year=car_year+[int(date[0:4])]
        
cars['year']=car_year
cars=cars.drop(columns=['DATE'])
cars=cars.rename(columns={'TOTALSA':'tot_population'})
cars=cars[['year','tot_population']]
cars=cars[cars['year']>=2002]
cars=cars.reset_index(drop=True)

In [42]:
hev.columns=['year','hybrids']
PEVs=pev.transpose().iloc[2:9,-1].reset_index()
PEVs.columns=['year','plug-ins']

vehicles=hev.merge(PEVs, on='year', how='outer')
vehicles = vehicles[vehicles['year']>=2002]
vehicles=vehicles.reset_index(drop=True)

veh_pops=pop['tot_population'][pop.year.isin(vehicles.year)].reset_index(drop=True) 
vehicles['percent_hybrid']=vehicles['hybrids']/veh_pops
vehicles['percent_plug-in']=vehicles['plug-ins']/veh_pops
vehicles=vehicles.drop(columns=['hybrids','plug-ins'])

,year,percent_hybrid,percent_plug-in
0,2002,0.000125,NaN
1,2003,0.000164,NaN
2,2004,0.000288,NaN
3,2005,0.000710,NaN
4,2006,0.000847,NaN


In [43]:
waste=waste.transpose()[['Products - Total Materials','Total MSW Generated - Weight']].reset_index()
waste.columns=['year','tot_recycled','total_MSW']
waste['percent_recycled']=waste['tot_recycled']/waste['total_MSW'] * 100
waste = waste.drop(columns=['tot_recycled'])
waste['year']=[float(x) for x in waste['year']]
waste = waste[waste['year']>=2002]

waste=waste.reset_index(drop=True)
waste_pops=pop['tot_population'][pop.year.isin(waste.year)].reset_index(drop=True) 
waste['MSW_per_capita']=waste['total_MSW']/waste_pops
waste=waste.drop(columns=['total_MSW'])

In [47]:
electric=(elec[['Year','Residential']]
          .groupby('Year')
          .sum()
          .reset_index())
electric = electric.rename(columns={'Year':'year'})
electric = electric[electric['year']>=2002]
electric=electric.reset_index(drop=True)
elec_pops=pop['tot_population'][pop.year.isin(electric.year)].reset_index(drop=True) 
electric['residential_electric_per_capita']=electric['Residential']/elec_pops
electric=electric.drop(columns=['Residential'])

In [9]:
DFs=[UPT,VRM,VRH]
Noms=['public_transit_trips','public_transit_miles','public_transit_hours']
year=[]
transport=pd.DataFrame()


for (df,nom) in zip(DFs, Noms):
    print(nom)
    dates = list(df.columns)
    del dates[0]
    melted = df.melt(id_vars=['Modes'], value_vars=dates, var_name='Date', value_name=nom)

    if nom=='public_transit_trips':
        transport['Modes']=melted['Modes']
        
#         print('Get years')
        for word in melted['Date']:
            if len(year)==0:
                print('First Element')
            elif year[len(year)-1][2:]!=word[3:]:
                print(word[3:])
            else:
                pass
            year=year+['20'+word[3:]]

        transport['year']=[ float(x) for x in year ]

    transport[nom]=melted[nom]
    
transport = transport.groupby(['year']).mean().reset_index()
transport = transport[transport['year']>=2002]
transport=transport.reset_index(drop=True)

UPT
First Element
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
VRM
VRH


KeyError: 'year'

Alaska Railroad (AR)
Cable car (CC)
Commuter rail (CR)
Heavy rail (HR)
Hybrid rail (YR)
Inclined plane (IP)
Light rail (LR)
Monorail/Automated guideway transit (MG)
Streetcar (SR)
Aerial Tramway (TR)
Bus (MB)
Bus rapid transit (RB)
Commuter bus (CB)
Demand Response (DR)
Demand taxi (DT)
Ferryboat (FB)
Jitney (JT)
Publico (PB)
Trolleybus (TB)
Vanpool (VP)

In [56]:
data_frames = [transport,meat,vehicles,waste,electric,fuel]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['year'],
                                            how='outer'), data_frames)
df_merged.to_csv(r'data\Cleaned and Combined.csv')